<a href="https://colab.research.google.com/github/chainiqedu/chainiqedu/blob/main/stETH_use_cases_lido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![An image](https://moonbeam.network/wp-content/uploads/2021/09/lido-500.png)

# Free Square Lido Bounty: stETH Analysis and Use Cases

### Contact Details

**Discord:** chainiqedu#0820  
**Twitter:** @chainiqedu 

### Overview

This notebook is a bounty submission for Lido on flipsidecrypto.  

This notebook performs an analysis of the stETH token and it's use cases across the DeFi eco-system.  

The notebook is split into sections which are:  



1.   stETH High Level Overview
2.   stETH Token Holders - Who/What types of addresses are holding stETH?
3.   stETH Use Cases Overview - What is stETH being used for?
4.   stETH Dex Overview - What dexes are using stETH and what for?
5.   stETH DeFi Overview  - What DeFi protocols are using stETH and what for?  

### Notes  

Each section contains a definition of the metrics used, a description of the data sources and the key insights and takeaways from the metrics/graphs.  

All of the data used in this notebook has been sourced from flipsidecrypto.  

For more details on Lido see here: https://lido.fi/faq

### Terms Definitions



*   Minting - The creation of a new token, e.g When Eth is deposited to Lido, stETH is minted (that is, created and given to the user who deposited the Eth)
*   Labels - For label types e.g dex, cex and definitions see here: https://docs.flipsidecrypto.com/our-data/data-models/labels
*   Others label type is any address that didn't have a defined label by the flipside crypto data










In [ ]:
#@title
# Import libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
#@title
steth_circulating_supply_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/877b22b9-9598-4746-a6bf-8884d1f4c9f8/data/latest')
steth_daily_close_prices_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/d90968fb-d0b3-428b-b193-b5bfe194a07e/data/latest')
steth_holders_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/b8741eef-e611-4987-b988-99f350c796c0/data/latest')
steth_holders_by_type = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/137b956b-03d8-474f-9958-73fde1fd2350/data/latest')
steth_stakers = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/c7fe98bd-8f02-4819-9bbe-1a026b981579/data/latest')
steth_mint_stake_transactions_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/7210819f-48e1-443d-a6b8-3ee6278988af/data/latest')
steth_erc20_balances_holders_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/07632592-5869-4efd-b1e1-4dcad8e5fe62/data/latest')

In [ ]:
#@title
# Total Circulating Supply Metrics
steth_circulating_supply_df['date_formatted'] = steth_circulating_supply_df['date'].dt.date
t_and_t_minus_one_values_df = steth_circulating_supply_df.iloc[-2:]
latest_circulating_supply_value = t_and_t_minus_one_values_df.tail(1)
t_minus_one_circulating_supply_value = t_and_t_minus_one_values_df.head(1)

# Latest stETH Price Metrics
latest_steth_close_prices = steth_daily_close_prices_df.iloc[-2:]
latest_st_eth_close_price_value = latest_steth_close_prices.tail(1)
t_minus_one_st_eth_close_price_value = latest_steth_close_prices.head(1)

# Distinct current stETH holders
distinct_num_steth_holders = steth_holders_df['address'].nunique()

# Number of minting/staking transactions
steth_mint_stake_transactions_df['date_formatted'] = steth_mint_stake_transactions_df['date'].dt.date

# Number of holders from erc20 balances per day
steth_erc20_balances_holders_df['date_formatted'] = steth_erc20_balances_holders_df['date'].dt.date

# stETH High Level Overview

### Description  

This section provides a high level overview and some fundamental metrics regarding the stETH token.  

### Key Insights  



*   The total circulating supply of stETH is increasing over time indicating that more ETH is being staked into the protocol  
*   The number of addresses holding stETH is increasing over time indicating more user activity for Lido  
*   Dex addresses are the highest holders of stETH by category  
*   The number of stETH minting transactions in September '21 saw an all time high spike and then a level off  
*   This level off however is a step increase higher than any previous stable level for the protocol indicating that there is a new higher stable level of users staking in Lido  



### Metric Definitions and Data Sources   

**Metric Name:** Total Circulating Supply stETH   
**Definition:**  Total amount of stETH currently in circulation. Calculated by summing every address that currently holds stETH.  

**Metric Name:** Latest stETH Price  
**Definition:**  The latest price of stETH in USD    

**Metric Name:** Number of stETH Holders  
**Definition:**  The number of distinct addresses holding a non-zero balance of stETH   

**Metric Name:** Total Circulating Supply stETH Time Series     
**Definition:**  Total amount of stETH that was in circulation at the specified date on the x-axis. Shows the trend of stETH in circulation.  

**Metric Name:** stETH Price ($) Time Series   
**Definition:**  Shows the price history of stETH. The daily price of stETH is considered the price logged in the at 23:00 hours on a given date in the ethereum.token_price_hourly table.   

**Metric Name:** Number of Addresses holding stETH     
**Definition:**  Shows the number of addresses holding stETH  

**Metric Name:** Volume of stETH Minted Time Series      
**Definition:**  Shows the number of stETH tokens minted per day.  

**Metric Name:** Number of stETH Minting Transactions     
**Definition:**  Shows the number of stETH minting transactions per day.   

**Metric Name:** Number of stETH by Address Category    
**Definition:**  Shows the number of stETH each category of address is holding.   

**Data Sources:** ethereum.udm_events, ethereum.erc20_balances (See cell 3 for queries and API data))  

In [ ]:
#@title
# steth_holders
steth_holders_label_holdings = steth_holders_by_type.groupby(['label_type_formatted']).sum().sort_values(by='total_holdings', ascending=False)
steth_holders_labels = steth_holders_label_holdings.index.values
steth_holders_values = steth_holders_label_holdings['total_holdings'].values


fig = make_subplots(
    rows=3, cols=3,
    specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
          [{"type": "scatter"}, {"type": "scatter"}, {"type": "bar"}],
          [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("Total Circulating Supply stETH", 
                    "Latest stETH Price ($)", 
                    "Number of stETH Holders", 
                    "Total Circulating Supply stETH Time Series",
                    "stETH Price ($) Time Series",
                    "Number of Addresses Holding stETH Time Series",
                    "Volume of stETH Minted Time Series",
                    "Number of stETH Minting Transactions Time Series",
                    "Number of stETH by Address Category")
)

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = latest_circulating_supply_value['total_cumulative_circulating_supply'].values[0],
    delta = {'reference': t_minus_one_circulating_supply_value['total_cumulative_circulating_supply'].values[0], 'relative': True}),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = latest_st_eth_close_price_value['price'].values[0],
    number = {'prefix': "$"},
    delta = {'reference': t_minus_one_st_eth_close_price_value['price'].values[0], 'relative': True}),
row=1, col=2)

fig.add_trace(go.Indicator(
    mode = "number",
    value = distinct_num_steth_holders),
row=1, col=3)

fig.add_trace(go.Bar(
    x = steth_circulating_supply_df['date_formatted'].values,
    y = steth_circulating_supply_df['total_cumulative_circulating_supply'].values),
    row=2, col=1)

fig.add_trace(go.Scatter(
    x = steth_daily_close_prices_df['hour'].values,
    y = steth_daily_close_prices_df['price'].values),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = steth_erc20_balances_holders_df['date_formatted'].values,
    y = steth_erc20_balances_holders_df['num_holding_addresses'].values),
    row=2, col=3)

fig.add_trace(go.Bar(
    x = steth_circulating_supply_df['date_formatted'].values,
    y = steth_circulating_supply_df['total_supply_per_day'].values),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = steth_mint_stake_transactions_df['date_formatted'].values,
    y = steth_mint_stake_transactions_df['num_transactions_per_day'].values),
    row=3, col=2)

fig.add_trace(go.Bar(
    x = steth_holders_labels,
    y = steth_holders_values),
    row=3, col=3)

# Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Holder Category", row=2, col=3)

fig.update_yaxes(title_text="Total Circulating stETH Supply", row=2, col=1)
fig.update_yaxes(title_text="Price ($)", row=2, col=2)
fig.update_yaxes(title_text="Number of stETH", row=2, col=3)

# Row 3 axis labelling
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=3, col=3)

fig.update_yaxes(title_text="stETH Minted Token Volume", row=3, col=1)
fig.update_yaxes(title_text="Number of Transactions", row=3, col=2)
fig.update_yaxes(title_text="Number of stETH Holding Addresses", row=3, col=3)

fig.update_layout(height=800, 
                  showlegend=False, 
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

# stETH Staking Addresses vs stETH Holdings Addresses

### Description  

This section compares the addresses that receive the stETH tokens after staking and the addresses where the tokens end up being held.  

The purpose of this section is to determine at a high level, are users just holding their stETH once received?  
Or are they putting it to work and moving it around the DeFi/Crypto eco-system?  

### Key Insights



*   The most popular addresses to receive stETH from the minting address are others (any addresses that are not grouped into a category, assumption here is that these will be a majority of other contracts and user addresses not in the DeFi eco-system)
*   The latest holding addresses however show that Dex's are largest holders of stETH
*   The insight gained here is that users that receive stETH are not just holding it, they are putting it to work in Dex's and the DeFi eco-system
*   Almost 60% of stETH tokens are held in either Dex or DeFi addresses
*  This is indicating that stETH is a useful token in the Dex/DeFi eco-system however there is also untapped opportunity for another ~40% of users to put their stETH to work


### Metric Definitions and Data Sources  

**Metric Name:** stETH Stakers   
**Definition:**  Percentage breakdown of the volume of stETH tokens sent to each address type from the minting address g. E.g 96.2% of stETH upon minting was sent to an 'other' address from the minting address  
 

**Metric Name:** stETH Holders    
**Definition:**  Percentage breakdown of the latest volume of stETH tokens that are held by each address type. E.g 52.6% of stETH tokens are currently held by DEX address types  


**Data Sources:**   ethereum.udm_events

In [ ]:
#@title
# steth_holders
steth_holders_label_holdings = steth_holders_by_type.groupby(['label_type_formatted']).sum()
steth_holders_labels = steth_holders_label_holdings.index.values
steth_holders_values = steth_holders_label_holdings['total_holdings'].values

#steth_stakers
steth_stake_label_holdings = steth_stakers.groupby(['to_label_type_formatted']).sum()
steth_stake_labels = steth_stake_label_holdings.index.values
steth_stake_values = steth_stake_label_holdings['amount'].values


fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
)

fig.add_trace(go.Pie(
    labels=steth_stake_labels, values=steth_stake_values, 
    title = {"text": "stETH Stakers"},
    textinfo='label+percent',
    direction ='counterclockwise',
    sort=True,
    domain = {'x': [0, 0.10], 'y': [0.5, 1]},),
    row=1, col=1)

fig.add_trace(go.Pie(
    labels=steth_holders_labels, values=steth_holders_values,
    title = {"text": "stETH Holders"},
    textinfo='label+percent',
    direction ='counterclockwise',
    sort=True,
    domain = {'x': [0.10, 0.20], 'y': [0.5, 1]},
    ),
    row=1, col=2)

fig.update_traces(hole = .4,
                  hoverinfo ="label+percent")

fig.update_layout(height=600,
                  title_font_size=24,
                  title_xanchor='left')

fig.update(layout_showlegend=True)

fig.show()

# stETH DEX Use Cases  

### Description  

This section looks at stETH tokens stored at DEX addresses.  
What DEX's are holding stETH?  
What is the proportion breakdown stored at each DEX?  

### Key Insights  



*   The most popular and dominant exchange for stETH is Curve Finance by far



### Metric Definitions and Data Sources  

**Metric Name:** stETH DEX Holders   
**Definition:**  For stETH tokens that are held at DEX addresses, what is the percentage breakdown stored at each DEX? e.g 99.8% of stETH tokens stored at a DEX address are stored in Curve Finance  

**Data Sources:**  ethereum.udm_events

In [ ]:
#@title
steth_dex = steth_holders_by_type.loc[steth_holders_by_type['label_type_formatted'].isin(["dex"])]
steth_dex_holdings = steth_dex.groupby(['label']).sum()
dex_labels = steth_dex_holdings.index.values
dex_values = steth_dex_holdings['total_holdings'].values

fig = go.Figure(data=[go.Pie(
    labels=dex_labels, 
    values=dex_values, 
    hole=.3, 
    textinfo='label+percent',
    title = {"text": "stETH DEX Holders"},)])
fig.show()

![An image](https://miro.medium.com/max/1400/1*BuPs5y7sWXX36Z5ZLNgbrQ.png)

# stETH Dex In-Depth View: Curve Finance

### Description  

This section looks at the stETH Curve Finance Pool and it's activity.  
More details here: https://curve.fi/steth   

### Key Insights 



*   The stETH Curve Fi Pool Balance has shown an upward trend mostly over time - this indicates that there is a positive sentiment towards staking stETH in the curve pool  
*   This is shown by the fact that there are very few occassions where there is a net withdrawal from the pool, across time there is mostly always a positive increase in stETH in the pool  
*   The pool transaction activity is also growing showing there are more exchanges and swaps being completed by the stETH pool in Curve Finance  
*   As a result of this the Curve Fi trading volume in the pool has shown spikes of increase (Max in August 3rd of 63M)  
*   However even when not at that level the pool is having trading volume in the millions  
*   This is positive for users who stake their stETH Curve LP tokens as fee's for LP stakers are based on this trading volume, the higher the volume the more fee's stakers can expect to collect (see here for more details: https://curve.fi/steth/faq#:~:text=power%20of%20Curve.-,Fees,(members%20of%20the%20DAO).)  
*   This can be an added incentive to get users to provide liquidity into the stETH Curve Pool  

*   steCRV is the token given when liquidity is provided in the stETH-ETH Curve Pool representing the share of liquidity the user has provided
*   The total circulating supply of steCRV seems to be showing a slight trend upwards although has had periods of decrease but still has steCRV tokens supply in the millions
*   The number of steCRV holders is also increasing indicating the interest in supply stETH to the pool
*   The steCRV holders are dominated by 3 addresses which include the Lido Farming Reward Contract and a Balancer Vault address, yearn finance (again indicating the use across the DeFi eco-system) is also included in the holders
*   When these top 3 addresses are removed, you can see that there is a range of holders owning from 8% all the way down to very small % amounts



### Metric Definitions and Data Sources  

**Metric Name:** stETH Curve Fi Pool Balance Time Series    
**Definition:**  The current stETH balance of the stETH Curve Finance Pool shown over time      

**Metric Name:** Net stETH Deposited/Withdrawn from Curve Fi Pool    
**Definition:**  Shows whether there was a net deposit or withdrawals of stETH from the stETH Curve Finance pool over time. A Positive value means there was an overall inflow to the pool (the number of stETH in the pool increased) and a negative value means there was an overall outflow from the pool (the number of stETH in the pool decreased).    
  

**Metric Name:** stETH Curve Fi Pool Deposits Time Series    
**Definition:**  The number of stETH tokens deposited to the stETH Curve Finance Pool shown over time  
 

**Metric Name:** stETH Curve Fi Pool Withdraws Time Series     
**Definition:**  The number of stETH tokens withdrawn from the stETH Curve Finance Pool shown over time  
 

**Metric Name:** stETH Curve Fi Pool Swap Transactions Time Series    
**Definition:**  The number of swap transactions that occurred in the stETH Curve Finance Pool shown over time     
 

**Metric Name:** stETH Curve Fi Pool Trading Volume Time Series       
**Definition:**  The trading volume (sum of the USD value of the swaps that occurred) in USD in the stETH Curve Finance Pool shown over time e.g if 5 swaps worth 1000 dollars occurred for a day, then the Trading Volume would be 5000 dollars    

**Metric Name:** steCRV Total Circulating Supply Time Series        
**Definition:** The total circulating supply of steCRV shown over time  
 

**Metric Name:** Total Number of steCRV Holders        
**Definition:**  The total number of addresses holding steCRV shown over time   

**Metric Name:** steCRV Holders       
**Definition:**  Percentage breakdown of all addresses holding steCRV  

**Metric Name:** steCRV Holders (excluding largest 3 holders)       
**Definition:**  Percentage breakdown of all addresses holding steCRV excluding the top 3 addresses (See key insights for why)  


**Data Sources:** ethereum.udm_events, ethereum.erc20_balances

In [ ]:
#@title
# stETH Curve Finance Liqudity Pool
steth_curve_lp_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/1813873c-5ab6-4ba0-8ad0-50cddb618b7a/data/latest')
steth_curve_lp_df['date_formatted'] = steth_curve_lp_df['date'].dt.date
# stETH Deposits/Withdraws Curve Finance
steth_curve_deposit_withdraw_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/e5367ae3-c4fc-4ceb-813d-963594fde7f8/data/latest')
steth_curve_deposit_withdraw_df['date_formatted'] = steth_curve_deposit_withdraw_df['date'].dt.date
# steth despoits/withdraws isolated
steth_curve_deposits_df = steth_curve_deposit_withdraw_df[steth_curve_deposit_withdraw_df['type'] == 'DEPOSIT']
steth_curve_withdraws_df = steth_curve_deposit_withdraw_df[steth_curve_deposit_withdraw_df['type'] == 'WITHDRAWN']
# steth curve fi pool trading volume
steth_curve_trading_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/fa6f571c-510f-4886-89b3-0197313600dc/data/latest')
steth_curve_trading_df['date_formatted'] = steth_curve_trading_df['date'].dt.date
# steCRV balances
stecrv_balances_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/da491fac-d3a2-45be-8515-02e4c0bc6506/data/latest')
stecrv_balances_df['date_formatted'] = stecrv_balances_df['date'].dt.date
# latest steCRV balance distribution
stecrv_latest_dist_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/b4a9d2da-ae6a-49c6-8b4e-acc842d69825/data/latest')
stecrv_latest_dist_df['date_formatted'] = stecrv_latest_dist_df['date'].dt.date
# filter out Large addresses for Lido/Balancer Vault/Others
stecrv_latest_dist_filtered_df = stecrv_latest_dist_df.sort_values(by=['BALANCE'], ascending=False)[3:]

In [ ]:
#@title
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("stETH Curve Fi Pool Balance", 
                    "Net stETH Deposited/Withdrawn from Curve Fi Pool",
                    "stETH Curve Fi Pool Deposits",
                    "stETH Curve Fi Pool Withdraws",
                    "stETH Curve Fi Pool Transactions",
                    "stETH Curve Fi Pool Trading Volume ($)"))

fig.add_trace(go.Bar(
    x = steth_curve_lp_df['date_formatted'].values,
    y = steth_curve_lp_df['total_cumulative_steth'].values),
    row=1, col=1)

fig.add_trace(go.Bar(
    x = steth_curve_lp_df['date_formatted'].values,
    y = steth_curve_lp_df['net_steth_per_day'].values),
    row=1, col=2)

fig.add_trace(go.Bar(
    x = steth_curve_deposits_df['date_formatted'].values,
    y = steth_curve_deposits_df['amount'].values),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = steth_curve_withdraws_df['date_formatted'].values,
    y = steth_curve_withdraws_df['amount'].values),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = steth_curve_trading_df['date_formatted'].values,
    y = steth_curve_trading_df['num_transactions'].values),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = steth_curve_trading_df['date_formatted'].values,
    y = steth_curve_trading_df['trading_volume_usd'].values),
    row=3, col=2)

# Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)

fig.update_yaxes(title_text="stETH Balance", row=1, col=1)
fig.update_yaxes(title_text="Net stETH Deposited/Withdrawn", row=1, col=2)
fig.update_yaxes(title_text="stETH Deposit", row=2, col=1)
fig.update_yaxes(title_text="stETH Withdrawn", row=2, col=2)
fig.update_yaxes(title_text="stETH Curve Fi Pool Transactions", row=3, col=1)
fig.update_yaxes(title_text="stETH Curve Fi Pool Trading Volume", row=3, col=2)

fig.update_layout(height=800, 
                  showlegend=False, 
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

In [ ]:
#@title
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "pie"},
            {"type": "pie"}]],
    subplot_titles=("steCRV Total Circulating Supply", 
                    "Total Number of steCRV Holders",
                    "steCRV Holders",
                    "steCRV Holders (minus top 3)"))

fig.add_trace(go.Bar(
    x = stecrv_balances_df['date_formatted'].values,
    y = stecrv_balances_df['holdings'].values),
    row=1, col=1)

fig.add_trace(go.Bar(
    x = stecrv_balances_df['date_formatted'].values,
    y = stecrv_balances_df['num_holders'].values),
    row=1, col=2)

fig.add_trace(go.Pie(
    labels=stecrv_latest_dist_df['USER_ADDRESS'].values, 
    values=stecrv_latest_dist_df['BALANCE'].values, 
    textinfo='label+percent',
    direction ='counterclockwise',
    textposition = 'inside',
    sort=True),
    row=1, col=3)

fig.add_trace(go.Pie(
    labels=stecrv_latest_dist_filtered_df['USER_ADDRESS'].values, 
    values=stecrv_latest_dist_filtered_df['BALANCE'].values, 
    textinfo='label+percent',
    direction ='counterclockwise',
     textposition = 'inside',
    sort=True),
    row=1, col=4)

# Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)

fig.update_yaxes(title_text="steCRV Total Circulating Supply", row=1, col=1)
fig.update_yaxes(title_text="Total Number of steCRV Holders", row=1, col=2)

fig.update_layout(height=600, 
                  showlegend=False, 
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

# stETH DeFi Use Cases

### Description  

This section looks at stETH tokens stored at DeFi addresses.  
What DeFi applications or protocols are holding stETH?  
What is the proportion breakdown stored at each DeFi app or protocol?  

### Key Insights  



*   The only DeFi labelled address from the flipside data is Anchor protocol (see here for more details: https://blog.lido.fi/collateralising-beth-in-anchor-protocol/)



### Metric Definitions and Data Sources  

**Metric Name:** stETH DeFi Holders    
**Definition:**  For stETH tokens that are held at DeFi addresses, what is the percentage breakdown stored at each DeFi app? (In this case 100% is stored at Anchor protocol)  

**Data Sources:**  ethereum.udm_events

In [ ]:
#@title
steth_defi = steth_holders_by_type.loc[steth_holders_by_type['label_type_formatted'].isin(["defi"])]
steth_defi_holdings = steth_defi.groupby(['label']).sum()
defi_labels = steth_defi_holdings.index.values
defi_values = steth_defi_holdings['total_holdings'].values

In [ ]:
#@title
fig = go.Figure(data=[go.Pie(labels=defi_labels, values=defi_values, hole=.3, title = {"text": "stETH DeFi Holders"}, textinfo='label+percent')])
fig.show()

![An image](https://s2.coinmarketcap.com/static/img/coins/64x64/8857.png)

# stETH DeFi In-Depth View: Anchor Protocol

### Description  

This section examines the interaction between stETH and the Anchor protocol.  
More details: https://blog.lido.fi/collateralising-beth-in-anchor-protocol/

### Key Insights 



*   The stETH tokens deposited to the Anchor Vault is around the 100K mark or just below
*   This increased from August to now however has slowed down recently in terms of increase in balance
*   It is more common for there to be net deposits than withdrawals however from September onwards it hasn't been significant inflow of deposits
*   bETH is a token that represents stETH on the Terra blockchain, therefore this number represents how much stETH has been wrapped into bETH
*   bETH Total Rewards is showing the rewards generated by the Anchor protocol from the stETH wrapped to bETH on Terra
*   The rewards are positive usually sitting around just under $50K at the moment however it is clear that Curve Finance is still ahead on both stETH volume and rewards



### Metric Definitions and Data Sources  

**Metric Name:** stETH Anchor Vault Balance Time Series    
**Definition:**  The balance of stETH currently in the Anchor Vault shown over time    

**Metric Name:** Net stETH Anchor Vault Deposit/Withdraws Time Series    
**Definition:**  The net value of stETH deposited or withdrawn from the Anchor vault shown over time. A positive value indicates there was a net inflow (the number of stETH tokens in the vault increased) on a given day where as a negative value indicates there was a net outflow (the number of stETH tokens in the vault decreased).       

**Metric Name:** stETH Anchor Vault Deposits Time Series  
**Definition:**  The number of stETH tokens deposited into the Anchor vault each day    

**Metric Name:** stETH Anchor Vault Withdraws Time Series   
**Definition:**  The number of stETH tokens withdrawn from the Anchor vault each day    

**Metric Name:** bETH Total Circulating Supply Time Series   
**Definition:**  The total circulating supply of bETH shown over time    

**Metric Name:** bETH Total Rewards Time Series   
**Definition:**  The total rewards earned from depositing/staking bETH in the Anchor Protocol shown over time  


**Data Sources:**  ethereum.udm_events

In [ ]:
#@title
# DeFi Anchor Protocol Metrics
steth_anchor_defi_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/1cf157d0-ff66-4962-9324-f39fe0015a12/data/latest')
steth_anchor_defi_df['date_formatted'] = steth_anchor_defi_df['date'].dt.date
# DeFi Anchor Deposits/Withdraws Metrics
steth_anchor_deposits_withdraws = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/cc4ad3d8-8a7d-4045-ae9b-d6db86117bab/data/latest')
steth_anchor_deposits_withdraws['date_formatted'] = steth_anchor_deposits_withdraws['date'].dt.date
# DeFi Anchor Deposits and Withdraws Isolated
steth_anchor_deposits_df = steth_anchor_deposits_withdraws[steth_anchor_deposits_withdraws['type'] == 'DEPOSIT']
steth_anchor_withdraws_df = steth_anchor_deposits_withdraws[steth_anchor_deposits_withdraws['type'] == 'WITHDRAWN']
# DeFi bETH Token Total Supply from staked stETH
beth_anchor = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/00d7061e-33ba-4e21-916c-e023272cb348/data/latest')
beth_anchor['date_formatted'] = beth_anchor['date'].dt.date
# DeFi bETH Anchor Vault Rewards from staked stETH
beth_anchor_rewards = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/9bc2b5e3-84eb-4012-b400-da658d39d29f/data/latest')
beth_anchor_rewards['date_formatted'] = beth_anchor_rewards['date'].dt.date

In [ ]:
#@title
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("stETH Anchor Vault Balance", 
                    "Net stETH Anchor Vault Deposit/Withdraws",
                    "stETH Anchor Vault Deposits",
                    "stETH Anchor Vault Withdraws"))

fig.add_trace(go.Bar(
    x = steth_anchor_defi_df['date_formatted'].values,
    y = steth_anchor_defi_df['total_cumulative_steth'].values),
    row=1, col=1)

fig.add_trace(go.Bar(
    x = steth_anchor_defi_df['date_formatted'].values,
    y = steth_anchor_defi_df['net_steth_per_day'].values),
    row=1, col=2)

fig.add_trace(go.Bar(
    x = steth_anchor_deposits_df['date_formatted'].values,
    y = steth_anchor_deposits_df['amount'].values),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = steth_anchor_withdraws_df['date_formatted'].values,
    y = steth_anchor_withdraws_df['amount'].values),
    row=2, col=2)


# Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)

fig.update_yaxes(title_text="stETH Anchor Vault Balance", row=1, col=1)
fig.update_yaxes(title_text="stETH Anchor Vault Deposit/Withdraws", row=1, col=2)
fig.update_yaxes(title_text="stETH Anchor Vault Deposits", row=2, col=1)
fig.update_yaxes(title_text="stETH Anchor Vault Withdraws", row=2, col=2)

fig.update_layout(height=600, 
                  showlegend=False, 
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

# bETH Metrics

In [ ]:
#@title
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("bETH Total Circulating Supply", 
                    "bETH (Anchor) Total Rewards"))

fig.add_trace(go.Bar(
    x = beth_anchor['date_formatted'].values,
    y = beth_anchor['total_cumulative_circulating_supply'].values),
    row=1, col=1)

fig.add_trace(go.Bar(
    x = beth_anchor_rewards['date_formatted'].values,
    y = beth_anchor_rewards['total_rewards_usd'].values),
    row=1, col=2)


# Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)

fig.update_yaxes(title_text="bETH Total Circulating Supply", row=1, col=1)
fig.update_yaxes(title_text="bETH Total Rewards ($)", row=1, col=2)

fig.update_layout(height=600, 
                  showlegend=False, 
                  title_font_size=24,
                  title_xanchor='left')

fig.show()